<a href="https://colab.research.google.com/github/ftempesta/Data-Science-Online/blob/master/Tutorial_2_Clasificaci%C3%B3n.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tutorial 2: Clasificación

6 de mayo de 2021

1. Crea una copia de este notebook
2. Sigue las instrucciones del notebook y ejecuta los bloques de código en el orden en que aparecen.

## Preámbulo

Las celdas de código en este notebook sólo aceptan código en Python, pero hay ciertas "palabras clave" que permiten hacer otras cosas. Por ejemplo, con `!` podemos llamar a un comando del sistema.

En este caso, vamos a llamar a `pip` (Package Installer for Python) para instalar las librerías que vamos a usar en este laboratorio. 

Haz click en la celda para seleccionarla y luego presiona **Ctrl+Enter** o **Shift+Enter** para ejecutarla.

In [ ]:
!pip install scikit-learn pandas numpy matplotlib

## Cargar datos

Usaremos el dataset _iris_ disponible en scikit-learn.

![iris dataset](https://sebastianraschka.com/images/blog/2014/intro_supervised_learning/iris_petal_sepal_1.png)

In [ ]:
from sklearn.datasets import load_iris

iris = load_iris()

# la variable iris es un objeto con varios atributos
# podemos verlos escribiendo "iris." y luego TAB para que Colab nos muestre las 
# posibles completaciones

# mostramos las primeras 10 filas
iris.data[0:10]

In [ ]:
# listamos los atributos

iris.feature_names

['sepal length (cm)',
 'sepal width (cm)',
 'petal length (cm)',
 'petal width (cm)']

In [ ]:
# listamos las clases (10 primeras filas)

iris.target

In [ ]:
# target_names son los nombres de las clases
# clase 0 corresponde a 'setosa'
# clase 1 corresponde a 'versicolor'
# clase 2 corresponde a 'virginica'

iris.target_names

Usando la librería `pandas` para manipular datos podemos generar una vista más "agradable" de los datos:



In [ ]:
import pandas as pd

df = pd.DataFrame(iris.data, columns=iris.feature_names)
df['clase'] = iris.target_names[iris.target]

# df.sample muestrea n=10 filas
df.sample(n=10)

Podemos visualizar los datos usando una _scatterplot matrix_ (o matriz de dispersión) para ver cómo se
comportan los atributos:

In [ ]:
fig = pd.plotting.scatter_matrix(df, figsize=(10, 10))

Una notación estándar es llamar $X$ a la matriz que contiene a los datos e $y$ al vector que contiene el valor de la clase para cada fila en $X$. 

Es decir, $X$ tiene $N$ filas y $p$ columnas (donde $p$ es la cantidad de atributos) e $y$ es un vector de $N$ valores.

In [ ]:
# nos referiremos con 'X' a los datos y con 'y' a las clases
# nota que ambas son variables numéricas
# X es una 'matriz' (es una lista de listas de valores asociados a atributos)
# y es un vector (una lista de valores)

X = iris.data
y = iris.target

## Separar datos en train y test

**IMPORTANTE**: En este punto debemos crear nuestros conjuntos de entrenamiento y de test.

Tenemos dos opciones:

1. Hacemos un *muestreo aleatorio* de los datos para separarlos en train y test.
2. Hacemos un *muestreo *estratificado* con respecto a la clase. Es decir, nos aseguramos que tanto train y test tengan la misma distribución de clases.

In [ ]:
from sklearn.model_selection import train_test_split

# al dejar `random_state` con un valor fijo nos garantiza que
# bajo las mismas condiciones, los resultados serán los mismos
# En este caso, la partición será la misma al ejecutar este bloque
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42, stratify=y)

# Baseline

Lo primero que debemos considerar antes de entrenar o decidir con cuál modelo nos vamos a quedar es cuál es el "piso mínimo" que podemos esperar.

Un *baseline* es un modelo simple, básico, que incluso puede no aprender nada de los datos, pero que nos sirve para saber cuál es el rendimiento mínimo que podemos alcanzar con los datos.

En el siguiente código vamos a declarar un algoritmo de aprendizaje "dummy" o "tonto". En este caso, el modelo entrenado va a predecir cualquiera de las tres clases al azar.



In [ ]:
from sklearn.dummy import DummyClassifier

# declaramos el modelo dummy
# hasta ahora no hemos hecho nada con los datos
dc = DummyClassifier(strategy="uniform", random_state=42)

Ahora entrenamos un modelo. **IMPORTANTE**: El modelo se entrena con los datos de entrenamiento.

In [ ]:
# este método altera el modelo
dc.fit(X_train, y_train)

DummyClassifier(constant=None, random_state=42, strategy='uniform')

Ahora vemos cómo le fue a este modelo con los datos de test.

Le vamos a pasar los datos de prueba al modelo, el cual entregará una _predicción_ por cada uno de las filas de prueba.

In [ ]:
y_pred = dc.predict(X_test)

Finalmente comparamos el resultado de la predicción, `y_pred` con la respuesta correcta, `y_test`:

In [ ]:
# podemos ver el accuracy, precision, recall por cada clase
from sklearn.metrics import classification_report

# (ver https://datascience.stackexchange.com/questions/15989/micro-average-vs-macro-average-performance-in-a-multiclass-classification-settin
# diferencia entre macro y micro average)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.20      0.19      0.19        16
           1       0.27      0.24      0.25        17
           2       0.20      0.24      0.22        17

    accuracy                           0.22        50
   macro avg       0.22      0.22      0.22        50
weighted avg       0.22      0.22      0.22        50



Se observa que el accuracy es aproximadamente $0.22$.


# Entrenar un modelo

Entrenaremos un árbol de decisión con los datos:

In [ ]:
from sklearn import tree

clf = tree.DecisionTreeClassifier(random_state=42, criterion='entropy')
clf = clf.fit(X_train, y_train)

In [ ]:
# vemos cómo le fue

y_pred = clf.predict(X_test)

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        16
           1       0.83      0.88      0.86        17
           2       0.88      0.82      0.85        17

    accuracy                           0.90        50
   macro avg       0.90      0.90      0.90        50
weighted avg       0.90      0.90      0.90        50



Bastante mejor que al baseline.

Podemos ver el árbol resultante:

In [ ]:
import pydotplus 
from IPython.display import Image

dot_data = tree.export_graphviz(clf, 
                                feature_names=iris.feature_names, 
                                class_names=iris.target_names, 
                                filled=True, 
                                out_file=None) 

graph = pydotplus.graph_from_dot_data(dot_data) 
Image(graph.create_png())

# Estimar el error fuera de la muestra

Arriba usamos holdout (separar los datos en train y test) para evaluar el clasificador. 

Recuerda que un problema de holdout es que podemos tener buena o mala suerte y elegir una muestra muy particular cuando tenemos pocos datos.

Por ejemplo, ¿qué pasa si elegimos otra partición train/test?






In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=7, stratify=y)

clf = tree.DecisionTreeClassifier(random_state=42)
clf = clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))

Podemos ver que bajamos de un 95% a un 92% de accuracy. No es mucho, pero ilustra cómo puede cambiar el resultado si cambiamos la muestra.

Una forma más consistente de determinar el error fuera de la muestra es usando cross-validation.

## Cross-validation

Recuerda que CV divide los datos en $k$ partes iguales: entrena con $k-1$ partes y evalúa con la parte restante, y esto lo repite $k$ veces (una vez por cada parte a evaluar).
Esta técnica se conoce como $k$-folds cross-validation.

Vamos a hacer cross-validation de forma "manual" para ver cómo se comporta.

![](https://upload.wikimedia.org/wikipedia/commons/1/1c/K-fold_cross_validation_EN.jpg)


**NOTA**: Recuerda que CV puede usarse para dos propósitos muy relacionados entre sí:

1. Tener una estimación del error fuera de la muestra (error de test), como haremos en este ejemplo (en este ejemplo usamos todo `X` e `y`).

2. Elegir un modelo o ajustar parámetros de éste (para eso usamos `X_train` e `y_train`).

In [ ]:
from sklearn.model_selection import KFold

# este código tiene un problema!

# 5-fold cv
kf = KFold(n_splits=5)

accuracies = []

for train_index, test_index in kf.split(X):
    print("TRAIN:", train_index, "TEST:", test_index, sep='\n')
    print()
    
    X_train = X[train_index]
    X_test = X[test_index]
    
    y_train = y[train_index]
    y_test = y[test_index]

    clf = tree.DecisionTreeClassifier(random_state=42)
    clf.fit(X_train, y_train)
    
    y_pred = clf.predict(X_test)
    
    accuracy = accuracy_score(y_test, y_pred)
    accuracies.append(accuracy)

print()
print("Accuracy de cada fold:", accuracies)
print("Accuracy promedio: ", sum(accuracies) / len(accuracies))

## Medidas de evaluación

¿Qué pasa cuando tenemos clases desbalanceadas? Por ejemplo, si tenemos dos clases, y están en relación 9:1, ¿puede pensar en un clasificador que tenga al menos 90% de accuracy?

Podemos hacer un análisis más fino del rendimiento por clase usando más medidas de evaluación. Entre las más importantes se cuentan **Precision** y **Recall** (o Recuperación).

$$Precision = \frac{TP}{TP + FP}$$

$$Recall = \frac{TP}{TP+FN}$$

Nota que estas medidas son para una clase en particular. La medida para todo el dataset puede ser el promedio de la medida para cada clase.

Supongamos que tenemos varias clases, pero nos queremos enfocar en una en particular, digamos $c$:

- $TP$ corresponde a los True Positive, o Verdaderos Positivos, es decir, los aciertos del clasificador: cuando clasificamos una observación correctamente como $c$.
- $FP$, o False Positive, es cuando clasificamos incorrectamente una observación como $c$, cuando en verdad no lo era.
- $TN$, o True Negative, cuando clasificamos correctamente algo que no es $c$
- $FN$, False Negative, cuando clasificamos incorrectamente algo que no es $c$


Otra forma de entender Precision y Recall al clasificar una clase $c$ es como sigue: 

\begin{equation}
    \text{Precision}_c = \frac{\text{#(observations correctly classified as } c \text{)}}{ \text{#(observations classified as } c \text{)} }
\end{equation}

\begin{equation}
    \text{Recall}_c = \frac{\text{#(observations correctly classified as } c \text{)}}{ \text{#(observations of class } c\text{)} } 
\end{equation}


Adicionalmente defimos la _matriz de confusión_ para observar los errores del clasificador:



In [ ]:
from sklearn.metrics import confusion_matrix

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=54)

clf = tree.DecisionTreeClassifier(random_state=54)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

confusion_matrix(y_test, y_pred)

La matriz se interpeta de la siguiente forma:

| iris-setosa  | iris-versicolor  | iris-virginica  | ← clasificado como / clase real ↓ |
|:----:|:----:|:----:|--------------------:|
| 12 | 0  | 0  |              **iris-setosa** |
| 0  | 17  | 1  |              **iris-versicolor** |
| 0  | 2 | 18 |              **iris-virginica** |

# Overfitting

El objetivo de un clasificador es poder aprender lo más que pueda de los datos, y al mismo tiempo poder **generalizar** su "conocimiento" a datos nuevos que nunca ha visto. 

El _overfitting_ se produce cuando el clasificador pierde la capacidad de generalización. Esto se observa usualmente cuando el clasificador se _sobreajusta_ a los datos, impidiendo que pueda predecir correctamente datos nuevos que nunca ha visto.

Otro problema con el overfitting es que nos dificulta tener una buena estimación del error de un clasificador. Por ejemplo, ¿qué pasa si quiero estimar parámetros de un clasificador usando el conjunto de prueba?

¿En qué otros casos se produce este problema de estimación?


El ejemplo abajo ilustra el sobreajuste al conjunto de entrenamiento, impidiendo al clasificador generalizar a datos nuevos (a la función objetivo)

In [ ]:
# ejemplo obtenido de http://scikit-learn.org/stable/auto_examples/model_selection/plot_underfitting_overfitting.html

import numpy as np
import matplotlib.pyplot as plt
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score

np.random.seed(0)

def true_fun(X):
    return np.cos(1.5 * np.pi * X)

n_samples = 30
degrees = [1, 4, 15]

X = np.sort(np.random.rand(n_samples))
y = true_fun(X) + np.random.randn(n_samples) * 0.1


plt.figure(figsize=(14, 5))
for i in range(len(degrees)):
    ax = plt.subplot(1, len(degrees), i + 1)
    plt.setp(ax, xticks=(), yticks=())

    polynomial_features = PolynomialFeatures(degree=degrees[i],
                                             include_bias=False)
    linear_regression = LinearRegression()
    pipeline = Pipeline([("polynomial_features", polynomial_features),
                         ("linear_regression", linear_regression)])
    pipeline.fit(X[:, np.newaxis], y)

    # Evaluate the models using crossvalidation
    scores = cross_val_score(pipeline, X[:, np.newaxis], y,
                             scoring="neg_mean_squared_error", cv=10)

    X_test = np.linspace(0, 1, 100)
    plt.plot(X_test, pipeline.predict(X_test[:, np.newaxis]), label="Model")
    plt.plot(X_test, true_fun(X_test), label="True function")
    plt.scatter(X, y, edgecolor='b', s=20, label="Samples")
    plt.xlabel("x")
    plt.ylabel("y")
    plt.xlim((0, 1))
    plt.ylim((-2, 2))
    plt.legend(loc="best")
    plt.title("Degree {}\nMSE = {:.2e}(+/- {:.2e})".format(
        degrees[i], -scores.mean(), scores.std()))
plt.show()


# En conclusión

El flujo usual a la hora de entrenar un clasificador es el siguiente:

1. Tener datos. Verificar la fuente de los datos, la existencia de sesgos.
2. Separar datos en train y test set.
3. Realizar preprocesamiento y limpieza de datos en ambos sets, **de manera independiente**.
4. Determinar hiperparámetros del clasificador usando un conjunto de validación (holdout) o usando cross-validation.
5. Evaluar en el test set para tener una estimación del rendimiento real del clasificador.
6. Para el modelo final, usar todos los datos disponibles.

# Referencias

1. Tutorial scikit-learn de árboles de decisión. https://scikit-learn.org/stable/modules/tree.html#classification
1. Documentación de scikit-learn. http://scikit-learn.org/stable/index.html
2. Precision y Recall. https://en.wikipedia.org/wiki/Precision_and_recall
3. Machine Learning 101 (Google). https://docs.google.com/presentation/d/1kSuQyW5DTnkVaZEjGYCkfOxvzCqGEFzWBy4e9Uedd9k/preview?imm_mid=0f9b7e&cmp=em-data-na-na-newsltr_20171213#slide=id.g168a3288f7_0_58
4. WEKA (un programa visual con clasificadores y otras herramientas para ML). https://www.cs.waikato.ac.nz/ml/weka/
5. Curso de Data Mining con WEKA. https://www.cs.waikato.ac.nz/ml/weka/mooc/dataminingwithweka/